In [1]:
import pandas as pd
from astropy.io import fits
import numpy as np
from pathlib import Path
import glob

from dateutil.parser import parse

In [ ]:
# MUST BE RUN IN ASTROCONDA38

from fast_combine import fast_process, marckie_slice_combine_reduce

In [4]:
%mkdir processed
%mkdir processed_uncorrected

dark_hdu = fits.open(Path('./calibs/Master_DarkBias.fits'))
#SuperDarkBias = dark_hdu[0].data
SuperDarkBias = dark_hdu[0].data.astype(np.float32)


flatC_norm_hdu = fits.open(Path('./calibs/Master_Flat_C_norm.fits'))
SuperNormFlat_C = flatC_norm_hdu[0].data.astype(np.float32)


mkdir: processed: File exists
mkdir: processed_uncorrected: File exists


In [4]:
fast_process(glob.glob('MBAFiles/*/*'), './processed/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
reducing image 0 from file Brorfelde_4_20221111_224933_1.fits with timestamp offset 2000
Successfully sliced and combined combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-1

In [3]:
fast_process(glob.glob('MBAFiles/*/*'), './processed_uncorrected/', exp_time=10, chunk_size=6,
    darkbias=0, flat=1, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
combined_2022-11-12T04:27:31.0000__Brorfelde_2_20221111_195408_1.fits
reducing image 0 from file Brorfelde_2_20221111_195408_1.fits with timestamp offset 2000
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-12T05:46:20.0000__Brorfelde_3_20221111_211257_1.fits
reducing image 0 from file Brorfelde_3_20221111_211

In [ ]:
%mkdir combined_all

fast_process(glob.glob('MBAFiles/*/*'), './combined_all/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

In [3]:
# MUST BE RUN IN (astroconda3)

import alipy
from datetime import datetime



In [4]:
%mkdir aligned

def align_imgs(images_to_align, ref_image):

    identifications = alipy.ident.run(ref_image, list(images_to_align), visu=False)

    outputshape = alipy.align.shape(ref_image)

    for id in identifications: # list of the same length as images_to_align.
        if id.ok == True:
            alipy.align.affineremap(id.ukn.filepath, id.trans,
                shape=outputshape,
                makepng=True,
                outdir=f'aligned/')

            print("SUCCESS: %20s : %20s, flux ratio %.2f" % (id.ukn.name,
                                                        id.trans,
                                                        id.medfluxratio))

        else:
            print("FAILED: %20s" % (id.ukn.name))
    return identifications

mkdir: aligned: File exists


In [ ]:
imgs = pd.DataFrame(
    columns=('filepath', 'timestamp', 'ref_img'),
    #dtype=(str, datetime, str)
)

def _get_time(fp:str):
    name = Path(fp).name.split('__')[0].lstrip('combined_')
    return parse(name)
    #return datetime.datetime.strptime(name, 'combined_%Y-%m-%dT%X.0000')




imgs['filepath'] = glob.glob(f"processed/*.fits")

imgs['timestamp'] = imgs['filepath'].apply(_get_time)

## Filter cell

In [3]:
# FILTER CELL

night_1 = (imgs['timestamp'] >= parse('2022-11-11T05:51:00')) & (imgs['timestamp'] < parse('2022-11-12T00:00:00'))



night_2_group_1 = (
    (parse('2022-11-12T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-12T05:59:20'))
    )

night_2_group_2 = (
    (parse('2022-11-12T05:59:20') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-13T00:00:00'))
    )


night_3 = (
    (parse('2022-11-13T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-14T00:00:00'))
    )


In [ ]:
sum(night_1) # total number in group

In [ ]:
imgs.loc[night_1, 'ref_img'] = './processed/combined_2022-11-11T05:51:00.0000__Brorfelde_4_20221110_215057_0.fits'

imgs.loc[night_2_group_1, 'ref_img'] = './processed/combined_2022-11-12T04:36:31.0000__Brorfelde_2_20221111_195408_1.fits'
imgs.loc[night_2_group_2, 'ref_img'] = './processed/combined_2022-11-12T06:59:36.0000__Brorfelde_4_20221111_224933_0.fits'

imgs.loc[night_3, 'ref_img'] = ('./processed/combined_2022-11-13T02:40:26.0000__Brorfelde_1_20221112_180823_0.fits')

imgs.dropna(inplace=True)

imgs[night_3]

In [20]:
imgs.loc[night_2_group_2, 'ref_img'] = './processed/combined_2022-11-12T06:59:36.0000__Brorfelde_4_20221111_224933_0.fits'


In [21]:
from os.path import exists
# ensure all ref images are real files
imgs['ref_img'].apply(exists).all()

True

In [24]:
# perform alignment for the specified groups

for group in (night_1, night_2_group_1, night_3):
    identifications = align_imgs(imgs[group]['filepath'], imgs[group]['ref_img'].iloc[0])
    imgs.loc[group, 'ok'] = [id.ok for id in identifications]

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


##########  Preparing reference ...
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 8
EXT_NUMBER values found in catalog : 1
I've selected 8 sources
Making more quads, from quadlevel 0 ...
Made   35 quads from    8 stars (combi n=7 s=0 d=30.0)
Removing 0/35 duplicates
########## Processing processed/combined_2022-11-12T04:53:31.0000__Brorfelde_2_20221111_195408_1.fits
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 8
EXT_NUMBER values found in catalog : 1
I've selected 8 sources
Making more quads, from quadlevel 0 ...
Made   35 quads from    8 stars (combi n=7 s=0 d=30.0)
Removing 0/35 duplicates
Finding 4 best candidates among 35 x 35 (ukn x ref)
We have a maximum of 35 quad pairs
Cand  1 (dist.   0.00034842) : Rotation   +0.015886 [deg], scale 0.999776
Cand  2 (dist.   0.00041908) : Rotation   -0.009094 [deg], scale 0.999847
Cand  3 (dist.   0.00

In [35]:
#success rate night 3
sum(imgs[night_3]['ok']) / imgs[night_3].shape[0]

0.6229508196721312

In [2]:
imgs.to_csv('img_master_spreadsheet.csv')

NameError: name 'imgs' is not defined

# Photometry: must be run in astroconda38

In [34]:
import pandas as pd
from astropy.io import fits
import numpy as np
from pathlib import Path
import glob

from dateutil.parser import parse
import align_moving

from importlib import reload

from os.path import exists

In [45]:
align_moving = reload(align_moving)

In [66]:
# read in images table

imgs = pd.read_csv('img_master_spreadsheet.csv')

imgs['timestamp'] = imgs['timestamp'].apply(parse)

def _get_aligned_img_path(fp: str):
    return str(Path('./aligned/').joinpath(Path(fp).name.rstrip('.fits') + '_affineremap.fits'))

imgs['aligned_filepath'] = imgs['filepath'].apply(_get_aligned_img_path)
imgs['ok'] = imgs['aligned_filepath'].apply(exists)

okay = imgs['ok'] == True

imgs.sort_values('timestamp', inplace=True)

imgs
# PLEASE RUN THE FILTER CELL AFTER THIS (see above)

,Unnamed: 0,filepath,timestamp,ref_img,ok,aligned_filepath
227,247,processed/combined_2022-11-11T05:51:00.0000__B...,2022-11-11 05:51:00,./processed/combined_2022-11-11T05:51:00.0000_...,True,aligned/combined_2022-11-11T05:51:00.0000__Bro...
143,157,processed/combined_2022-11-11T05:52:00.0000__B...,2022-11-11 05:52:00,./processed/combined_2022-11-11T05:51:00.0000_...,True,aligned/combined_2022-11-11T05:52:00.0000__Bro...
333,364,processed/combined_2022-11-11T05:53:00.0000__B...,2022-11-11 05:53:00,./processed/combined_2022-11-11T05:51:00.0000_...,True,aligned/combined_2022-11-11T05:53:00.0000__Bro...
154,169,processed/combined_2022-11-11T05:54:00.0000__B...,2022-11-11 05:54:00,./processed/combined_2022-11-11T05:51:00.0000_...,True,aligned/combined_2022-11-11T05:54:00.0000__Bro...
574,624,processed/combined_2022-11-11T05:55:00.0000__B...,2022-11-11 05:55:00,./processed/combined_2022-11-11T05:51:00.0000_...,True,aligned/combined_2022-11-11T05:55:00.0000__Bro...
...,...,...,...,...,...,...
375,407,processed/combined_2022-11-13T06:59:17.0000__B...,2022-11-13 06:59:17,./processed/combined_2022-11-13T02:40:26.0000_...,False,aligned/combined_2022-11-13T06:59:17.0000__Bro...
19,22,processed/combined_2022-11-13T07:00:17.0000__B...,2022-11-13 07:00:17,./processed/combined_2022-11-13T02:40:26.0000_...,False,aligned/combined_2022-11-13T07:00:17.0000__Bro...
441,481,processed/combined_2022-11-13T07:01:17.0000__B...,2022-11-13 07:01:17,./processed/combined_2022-11-13T02:40:26.0000_...,False,aligned/combined_2022-11-13T07:01:17.0000__Bro...
519,564,processed/combined_2022-11-13T07:02:17.0000__B...,2022-11-13 07:02:17,./processed/combined_2022-11-13T02:40:26.0000_...,False,aligned/combined_2022-11-13T07:02:17.0000__Bro...


# Photometry

In [71]:
# click on image to print its coordinates and timestamp!
# MAKE SURE TO RUN pip install mplcursors
%matplotlib tk

align_moving.get_coords_of_point(imgs.loc[night_3 & okay, 'aligned_filepath'].iloc[1])

515.4295693484006, 1435.3569531118235, '2022-11-13T02:15:26.0000'


In [60]:
# NIGHT 1

%matplotlib tk

FWHM = 13

photometry = align_moving.do_photometry_with_timestamps(
    imgs[okay][night_1]['aligned_filepath'],

    start_object_x=723.7147046501888,
    start_object_y=944.1470203183914,
    start_object_t='2022-11-11T06:14:00.0000',

    end_object_x=676.7723412773491,
    end_object_y=948.8662977890997,
    end_object_t='2022-11-11T06:01:00.0000', 

    aperture_=FWHM*3,
    annulus_inner=5*FWHM,
    annulus_outer=9*FWHM,
    dry_run=True,

    reference_positions=[(3453, 4545), (576, 45)] # CHANGE THIS
 
)

photometry

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_23590/743432291.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_1]['aligned_filepath'],


,target_subtracted_counts,object_subtracted_counts,num,differential,uncalibrated_object_mag,uncalibrated_target_mag,differential_mag


In [59]:
# Night 2 group 1

%matplotlib tk

FWHM = 10

photometry = align_moving.do_photometry_with_timestamps(
    imgs[okay][night_2_group_1]['aligned_filepath'],

    start_object_x=651,
    start_object_y=983,
    start_object_t='2022-11-12T02:49:48',

    end_object_x=1088,
    end_object_y=938,
    end_object_t='2022-11-12T04:51:31', 

    aperture_=FWHM*3,
    annulus_inner=5*FWHM,
    annulus_outer=9*FWHM,
    dry_run=True,

    reference_positions=[] # CHANGE THIS

)

photometry

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_23590/218010773.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_2_group_1]['aligned_filepath'],


,target_subtracted_counts,object_subtracted_counts,num,differential,uncalibrated_object_mag,uncalibrated_target_mag,differential_mag


In [68]:
# Night 3

#1211.797141170518, 858.9736043177603, '2022-11-13T06:35:57.0000'

#617.4767807949628, 932.9911452184181, '2022-11-13T03:51:33.0000'

%matplotlib tk

FWHM = 17

photometry = align_moving.do_photometry_with_timestamps(
    imgs[okay][night_3]['aligned_filepath'],

    start_object_x=617.4767807949628,
    start_object_y=932.9911452184181,
    start_object_t='2022-11-13T03:51:33.0000',

    end_object_x=1211.797141170518,
    end_object_y=858.9736043177603,
    end_object_t='2022-11-13T06:35:57.0000', 

    aperture_=FWHM*3,
    annulus_inner=5*FWHM,
    annulus_outer=9*FWHM,
    dry_run=True,

    reference_positions=[] # CHANGE THIS

)

photometry

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_23590/1738472810.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_3]['aligned_filepath'],


,target_subtracted_counts,object_subtracted_counts,num,differential,uncalibrated_object_mag,uncalibrated_target_mag,differential_mag


In [8]:
imgs[okay][night_2_group_1]['aligned_filepath']

/var/folders/p8/xx7c8vpj7qj510bw5jdsr0800000gq/T/ipykernel_23049/2914660617.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imgs[okay][night_2_group_1]['aligned_filepath']


6      aligned/combined_2022-11-12T04:53:31.0000__Bro...
11     aligned/combined_2022-11-12T04:37:31.0000__Bro...
16     aligned/combined_2022-11-12T02:48:48.0000__Bro...
20     aligned/combined_2022-11-12T02:50:48.0000__Bro...
22     aligned/combined_2022-11-12T04:15:11.0000__Bro...
                             ...                        
595    aligned/combined_2022-11-12T03:19:08.0000__Bro...
598    aligned/combined_2022-11-12T04:23:11.0000__Bro...
599    aligned/combined_2022-11-12T04:18:11.0000__Bro...
602    aligned/combined_2022-11-12T02:45:48.0000__Bro...
603    aligned/combined_2022-11-12T04:00:11.0000__Bro...
Name: aligned_filepath, Length: 119, dtype: object

In [70]:
th = [[1, 2], [3, 4]]

pd.DataFrame(th, columns=['wow', 'wut']) + pd.DataFrame(th, columns=['wow', 'wut'])

,wow,wut
0,2,4
1,6,8
